# T5 Transformer Model 
Uses the Hugging Face pretrained `t5-large` model on our dataset.

*credit to this [writeup by Shivanandroy](https://shivanandroy.com/transformers-generating-arxiv-papers-title-from-abstracts/#notebooks)*

In [ ]:
import pandas as pd
import torch
import re

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

torch.cuda.get_device_name()

<br>

## Clean Data

In [ ]:
<br>df = pd.read_csv('songData.csv')
df = df.drop(columns = ['artist', 'link'])
df = df[['song','text']]

# remove '\r', '\n' as well as text within '[]', '()'
df['text'] = df['text'].replace(['\r', '\n'],'', regex=True)
df['text'] = df['text'].apply(lambda str1: "".join(re.split("\(|\)|\[|\]", str1)[::2]))
df['song'] = df['song'].replace(['\r', '\n'],'', regex=True)

df.head(3)

In [ ]:
df = df[['song','text']]
df.columns = ['target_text', 'input_text']
df = df.dropna()

In [ ]:
eval_df = df.sample(frac=0.2, random_state=101)
train_df = df.drop(eval_df.index)

In [ ]:
# sanity check
train_df.shape, eval_df.shape

# Training

In [ ]:
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df['prefix'] = "summarize"
eval_df['prefix'] = "summarize"


# T5 Training parameters
model_args = T5Args()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.max_seq_length = 128
model_args.train_batch_size = 16
model_args.num_train_epochs = 4
model_args.use_multiprocessing = False
model_args.best_model_dir = 'outputs/best_model'


# Create T5 Model
model = T5Model(model_name="t5-large", model_type="t5", args=model_args, use_cuda=True)

# Train T5 Model on new task
model.train_model(train_df)

# Evaluating

In [ ]:
<br># Evaluate T5 Model on new task
results = model.eval_model(eval_df)

# Predict with trained T5 model
#print(model.predict(["convert: four"]))